# 1. Kütüphanelerin Eklenmesi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#Uyarılar gizlendi
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import preprocess_input

#Sınıfların belirlenmesi

In [ ]:
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
class_names = [folder for folder in os.listdir(fish_dir) if os.path.isdir(os.path.join(fish_dir, folder))]
class_names

# 2. Veri Önişleme
Veri setimiz .png formatında fotoğraflardan oluştuğundan veriyi düzenlememiz ve Pandas DataFrame oluşturmamız gerekmektedir.

Veri ön işleme aşaması veri bilimi projelerinin en önemli aşamalarından biridir. Sonrasında sırada model kurma vardır.

In [ ]:
labels = []
paths = []
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if filename.endswith('.png') and 'GT' not in dir_name.split(): #if filename ends with png and directory doesn`t contain GT
            labels.append(os.path.split(dir_name)[-1])                 #Append the directory name to label
            paths.append(os.path.join(dir_name, filename))             #Append all the png files to path of that directory
            
# Pandas DataFrame oluşturulması
df = pd.DataFrame({'path': paths, 'label': labels})


# 2.1 Verinin İncelenmesi

In [ ]:
print("\n Toplam görüntü sayısı:", len(df))
print("\n Her türden kaç görüntü var:")
print(df['label'].value_counts())

In [ ]:
print("Veri setinin boyutu:", df.shape)
print("\n Sütun bilgileri:")
df.info()

print("\n İlk 5 satır:")
print(df.head(5))

print("\n Balık türü sayısı:", df['label'].nunique())

In [ ]:
plt.figure(figsize=(12, 12))
for i, label in enumerate(df['label'].unique()):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(df[df['label'] == label].iloc[0, 0]))
    plt.title(label)
    plt.axis('off')
plt.show()

# 2.2 Veri Setinin Train & Test Olarak Ayrılması

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

test_size=0.2 parametresi ile veri setinin %20'sini test kümelerine ayırdık. random_state=42 parametresiyle rastgele bölme işleminin tekrarlanabilirliğini sağladık.

In [ ]:
x_train = np.array([x for x in x_train])
y_train = np.array([y for y in y_train])
x_test = np.array([x for x in x_test])
y_test = np.array([y for y in y_test])


#One-Hot Encoding, kategorik değişkenlerin ikili vektörler olarak temsil edilmesini sağlar.

encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# 3. Modelin Eğitilmesi

In [ ]:
model = tf.keras.models.Sequential([
  # inputs 
  tf.keras.layers.Flatten(input_shape=(225, 225, 3)),
  # hiddens layers
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  # output layer
  tf.keras.layers.Dense(9,activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
# Modelin derlenmesi
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # val_loss yerine val_accuracy kullanıyoruz
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  # val_loss yerine val_accuracy kullanıyoruz
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

In [ ]:
history = model.fit(
    train_images,
    epochs=20,
    validation_data=val_images,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# 4. Modelin Değerlendirilmesi

# 4.1 Accuracy Hesabı & Kayıp Fonksiyonu

In [ ]:
plt.plot(results.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(results.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()

plt.plot(results.history['loss'], label='Eğitim Kaybı')
plt.plot(results.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()
plt.show()

# 4.2 Test Sonuçları

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test acc: {test_acc}")
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print(classification_report(y_true_labels, y_pred_labels))
sns.heatmap(confusion_matrix(y_true_labels, y_pred_labels), annot=True, fmt='d', cmap='Reds')
plt.show()